<a href="https://colab.research.google.com/github/qingqingye/impact-of-network-6607-project-topic2/blob/main/dataclean_lda_6607project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline

In [ ]:
from operator import truediv
from sqlite3 import DataError
import pandas as pd


df = pd.read_csv(r'Expt3-ece6607-2022-fall modified.csv')
dataframe = df[df['Your suggestions'].str.contains('\n\n')]
dataframe = dataframe.reset_index(drop = True)

dataframe['Before'] = dataframe['Your suggestions'].str.split('\n\n', expand = True)[0]
dataframe['After'] = dataframe['Your suggestions'].str.split('\n\n', expand = True)[1]
dataframe["Before"]= dataframe["Before"].str.split("\n", expand = False)
dataframe["After"]= dataframe["After"].str.split("\n", expand = False)

dataframe.to_csv('clean.csv', index=False)

In [ ]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [ ]:
from sklearn.utils import as_float_array
# test_df = pd.read_csv('test.csv')
# print(test_df)

# reindexed_data = test_df['Before']
# reindexed_data.index = test_df['Timestamp']


dict = {'Type': [],
        'Priority': [],
        'Suggestion': [],
        'index_num': []}
for i in range(dataframe.shape[0]):
  before_list = dataframe.loc[i, 'Before']
  after_list = dataframe.loc[i, 'After']
  min_len = min(len(before_list), len(after_list))
  for i, suggestion in enumerate(before_list[: min_len]):
    priority = i + 1
    if len(suggestion) == 0:
      continue
    dict['Type'].append(0)
    dict['Priority'].append(priority)
    dict['Suggestion'].append(suggestion)
  for i, suggestion in enumerate(after_list[: min_len]):
    priority = i + 1
    if len(suggestion) == 0:
      continue
    dict['Type'].append(1)
    dict['Priority'].append(priority)
    dict['Suggestion'].append(suggestion)
dict['index_num'] = range(len(dict['Type']))

indexed_data = pd.DataFrame(dict)
indexed_data.dropna()
indexed_data.to_csv('indexed_data.csv', index=False)
print(indexed_data.shape[0])

# count_vectorizer = CountVectorizer(stop_words='english')
# words, word_values = get_top_n_words(n_top_words=10,
#                                      count_vectorizer=count_vectorizer, 
#                                      text_data=reindexed_data)

# fig, ax = plt.subplots(figsize=(16,8))
# ax.bar(range(len(words)), word_values);
# ax.set_xticks(range(len(words)));
# ax.set_xticklabels(words, rotation='vertical');
# ax.set_title('Top words in headlines dataset (excluding stop words)');
# ax.set_xlabel('Word');
# ax.set_ylabel('Number of occurences');
# plt.show()

519


In [ ]:
df_indexed = pd.read_csv('indexed_data.csv')
reindexed_data = df_indexed['Suggestion']
reindexed_data.index = df_indexed['index_num']
# reindexed_data.index = 

In [ ]:
small_count_vectorizer = CountVectorizer(stop_words='english', max_features=40000)
small_text_sample = reindexed_data.sample(n=500, random_state=0).values

print('Headline before vectorization: {}'.format(small_text_sample[0]))

small_document_term_matrix = small_count_vectorizer.fit_transform(small_text_sample)

print('Headline after vectorization: \n{}'.format(small_document_term_matrix[0]))

Headline before vectorization: I think that the CULC roof should be open everyday.
Headline after vectorization: 
  (0, 597)	1
  (0, 141)	1
  (0, 507)	1
  (0, 393)	1
  (0, 193)	1


In [ ]:
n_topics = 15

In [ ]:
# Define helper functions
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

In [ ]:
# Define helper functions
def get_top_n_words_(n, keys, document_term_matrix, count_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [ ]:
lda_model = LatentDirichletAllocation(n_components=n_topics, learning_method='online', 
                                          random_state=0, verbose=0)
lda_topic_matrix = lda_model.fit_transform(small_document_term_matrix)
lda_keys = get_keys(lda_topic_matrix)
lda_categories, lda_counts = keys_to_counts(lda_keys)
top_n_words_lda = get_top_n_words_(8, lda_keys, small_document_term_matrix, small_count_vectorizer)

for i in range(len(top_n_words_lda)):
    print("Topic {}: ".format(i+1), top_n_words_lda[i])

Topic 1:  increased campus pay gra opportunities employment number student
Topic 2:  students free mental health campus access help provide
Topic 3:  exams students open research book conceptual understanding able
Topic 4:  options food campus course ece students better cheaper
Topic 5:  student events fees lower library job support books
Topic 6:  major students hands experience courses mindset crime improve
Topic 7:  registration discounts course gas foods costco process early
Topic 8:  course mode project person remote availability lectures wish
Topic 9:  lower classes better housing students rent food quality
Topic 10:  parking reduce campus fee traffic free affordable fees
Topic 11:  discounted taxes employees speed improve student fi wi
Topic 12:  opportunities ta assignments ra assignment exams weightage funding
Topic 13:  courses students semester grad especially international create campus
Topic 14:  better campus transportation affordable meal coverage flexible plans
Topic 15